In [140]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [141]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [142]:
def finep():

    finep = pd.DataFrame()

    titulos_proyectos = []
    fechas_publicación_proyectos = []
    fechas_cierre_proyectos = []
    fuentes_recurso_proyectos = []
    publicos_objetivo_proyecto = []
    temas_proyectos = []
    links_proyectos = []
    descripciones = []
    publicos = []
    pdfs_proyectos = []

    ###########
    base = 'http://www.finep.gov.br/chamadas-publicas?situacao=aberta'

    parser_pagina = parser(base)
    pag_max = parser_pagina.xpath('//p[@class="counter pull-right"]/text()')[0].strip()[-1]
    pag_max

    #########

    n = 0

    while n <= int(pag_max)-1:

        if n == 0:
            page = 'http://www.finep.gov.br/chamadas-publicas?situacao=aberta'
        else:
            page = 'http://www.finep.gov.br/chamadas-publicas?situacao=aberta&start=' + str(n) + '0'


        parser_pagina = parser(page)

        titulos = parser_pagina.xpath('//div[@id="conteudoChamada"]//a/text()')

        links = parser_pagina.xpath('//div[@id="conteudoChamada"]//a/@href')
        links = ['http://www.finep.gov.br' + i for i in links]

        fechas_publicacion = parser_pagina.xpath('//div[@id="conteudoChamada"]//div[@class="data_pub div"]/span/text()')
        
        fechas_cierre = parser_pagina.xpath('//div[@id="conteudoChamada"]//div[@class="prazo div"]/span/text()')

        fuentes_recurso = parser_pagina.xpath('//div[@id="conteudoChamada"]//div[@class="fonte div"]/span/text()')
        fuentes_recurso = [i.strip() for i in fuentes_recurso]


        temas = parser_pagina.xpath('//div[@id="conteudoChamada"]//div[@class="tema div"]/span/text()')
        temas = [i.strip() for i in temas]

        titulos_proyectos += titulos
        fechas_publicación_proyectos+=fechas_publicacion
        fechas_cierre_proyectos+=fechas_cierre
        fuentes_recurso_proyectos+=fuentes_recurso
        temas_proyectos+=temas
        links_proyectos+=links

        n+=1

    ########################

    for link in links_proyectos:

        parser_proyecto = parser(link)
        texts = parser_proyecto.xpath('//div[@class="group desc"]/div[@class="text"]//text()')

        # Descripción
        texts = parser_proyecto.xpath('//div[@class="group desc"]/div[@class="text"]//text()')

        descripcion = ''
        for text in texts:
            text = text.strip()
            descripcion = descripcion + text + ' '
        descripciones.append(descripcion)
        # Público objetivo
        
        publico = ''
        try:
            publico = parser_proyecto.xpath('//span[@class="tag"]/text()')
            publicos.append(publico[0])

        except:
            publico = ''
            publicos.append(publico)

        # Estado
        pdfs = parser_proyecto.xpath('//tbody//a//@href')
        pdfs = ['http://www.finep.gov.br' + i for i in pdfs]

        pdfs_proyecto = ''
        for pdf in pdfs:
            pdfs_proyecto = pdfs_proyecto + pdf + ', '

        pdfs_proyecto.strip(', ')

        pdfs_proyectos.append(pdfs_proyecto)

    finep['Título'] = titulos_proyectos
    finep['Descripción'] = descripciones
    finep['Fecha públicación'] = fechas_publicación_proyectos
    finep['Fecha cierre'] = fechas_cierre_proyectos
    finep['Público Objetivo'] = publicos
    finep['Fuente Recurso'] = fuentes_recurso_proyectos
    finep['Tema'] = temas_proyectos
    finep['Link'] = links_proyectos
    finep['Pdfs'] = pdfs_proyectos
    

    return finep



In [143]:
finepT = finep()
finepT.to_excel('finep_brasil.xlsx')